# Efficient Yelp API Calls (Core)
- Clint Atterberry
- 8/24/22

For this assignment, you will be working with the Yelp API.

As before, you will use the Yelp API to search your favorite city for a cuisine type of your choice.

Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

Save your notebook, commit the change to your repository and submit the repository URL for this assignment.

## Setup

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
with open('/Users/oneda/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
# set our API call parameters and filename before the first call
LOCATION = 'Chicago, IL, 60654'
TERM = 'Pizza'

## JSON File Prep

In [4]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = f"Data/results_in_progress_Pizza.json"
JSON_FILE

'Data/results_in_progress_Pizza.json'

In [5]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_Pizza.json already exists.


In [6]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 1000 previous results found.


## API Results

In [11]:
# use our yelp_api variable's search_query method to perform our API call
# todo: account for limit offset
results = None

if n_results < 1000:
    try:
        results = yelp_api.search_query(location=LOCATION,
                                           term=TERM,
                                           offset=n_results)
        results.keys()
        ## How many results total?
        total_results = results['total']
        total_results
    except:
        pass


In [18]:
## How many did we get the details for?
if results:
    results_per_page = len(results['businesses'])
    results_per_page



In [19]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
if results:
    n_pages = math.ceil((results['total']-n_results)/ results_per_page)
    n_pages

## Write to JSON file

In [20]:
# join new results with old list with extend and save to file
if results:
    previous_results.extend(results['businesses'])  
    with open(JSON_FILE,'w') as f:
         json.dump(previous_results,f)

In [23]:
if results:
    from tqdm.notebook import tqdm_notebook
    import time
    for i in tqdm_notebook(range(n_pages)):
        # adds 200 ms pause
        time.sleep(.2) 


In [25]:
if results:
    for i in tqdm_notebook( range(1,n_pages+1)):
        time.sleep(.2)
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
        ## save number of results for to use as offset
        n_results = len(previous_results)
        ## use n_results as the OFFSET 
        results = yelp_api.search_query(location=LOCATION,
                                        term=TERM, 
                                        offset=n_results)

        ## append new results and save to file
        previous_results.extend(results['businesses'])

    #     display(previous_results)
        with open(JSON_FILE,'w') as f:
            json.dump(previous_results,f)

In [26]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,8vFJH_paXsMocmEO_KAa3w,lou-malnatis-pizzeria-chicago,Lou Malnati's Pizzeria,https://s3-media3.fl.yelpcdn.com/bphoto/FXGmfo...,False,https://www.yelp.com/biz/lou-malnatis-pizzeria...,6704,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.890357, 'longitude': -87.633704}","[delivery, pickup]",$$,"{'address1': '439 N Wells St', 'address2': '',...",+13128289800,(312) 828-9800,2240.253546
1,Bxd9Y2PsdvIOGIKFQh53RQ,giordanos-chicago-30,Giordano's,https://s3-media1.fl.yelpcdn.com/bphoto/OKxuR9...,False,https://www.yelp.com/biz/giordanos-chicago-30?...,3215,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.88514, 'longitude': -87.62313}",[delivery],$$,"{'address1': '130 E Randolph', 'address2': '',...",+13126161200,(312) 616-1200,2529.782273
2,cxAkuwv7p4gm4Coixw7h9A,lou-malnatis-pizzeria-chicago-9,Lou Malnati's Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/vZVADY...,False,https://www.yelp.com/biz/lou-malnatis-pizzeria...,2856,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.871566, 'longitude': -87.627275}","[delivery, pickup]",$$,"{'address1': '805 S State St', 'address2': '',...",+13127861000,(312) 786-1000,1948.893306
3,TBoYnkPNFn9ASccYCYsJfw,nancys-pizza-chicago-15,Nancy's Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/r8eD2H...,False,https://www.yelp.com/biz/nancys-pizza-chicago-...,124,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.88331, 'longitude': -87.65269}","[pickup, delivery]",$$,"{'address1': '1000 W Washington', 'address2': ...",+13127339920,(312) 733-9920,1007.798803
4,09eEic9nbbSYMRizNfWrLQ,primos-chicago-pizza-pasta-chicago,Primos Chicago Pizza Pasta,https://s3-media4.fl.yelpcdn.com/bphoto/yBFoaB...,False,https://www.yelp.com/biz/primos-chicago-pizza-...,194,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.876834739546, 'longitude': -87...","[delivery, pickup]",$,"{'address1': '816 W Van Buren St', 'address2':...",+13122431052,(312) 243-1052,316.824958


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,Yt_zBJwh3PSGwlRr4EyA7w,el-paisano-tacos-chicago-2,El Paisano Tacos,https://s3-media2.fl.yelpcdn.com/bphoto/c15DQ2...,False,https://www.yelp.com/biz/el-paisano-tacos-chic...,134,"[{'alias': 'mexican', 'title': 'Mexican'}]",3.5,"{'latitude': 41.902702331543, 'longitude': -87...","[delivery, pickup]",$,"{'address1': '2429 W Division St', 'address2':...",+17732921002,(773) 292-1002,4429.925263
996,0jQB40_EoCL5a5PtnGzAjQ,jimmy-johns-chicago-53,Jimmy John's,https://s3-media2.fl.yelpcdn.com/bphoto/RkRC-F...,False,https://www.yelp.com/biz/jimmy-johns-chicago-5...,51,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",3.5,"{'latitude': 41.8923895, 'longitude': -87.6220...",[],$,"{'address1': '205 E Ohio St', 'address2': '', ...",+13122450010,(312) 245-0010,3075.234192
997,7Fh2VuePY1I8mGT8mW3Psw,potbelly-sandwich-shop-chicago-45,Potbelly Sandwich Shop,https://s3-media2.fl.yelpcdn.com/bphoto/EcxMar...,False,https://www.yelp.com/biz/potbelly-sandwich-sho...,44,"[{'alias': 'salad', 'title': 'Salad'}, {'alias...",3.0,"{'latitude': 41.905374, 'longitude': -87.6694092}","[delivery, pickup]",$,"{'address1': '1293 N Milwaukee Ave', 'address2...",+17736610182,(773) 661-0182,3769.659226
998,R9MFihPd2WlN6Ze7OcOdqQ,mirai-sushi-wicker-park-chicago,Mirai Sushi - Wicker Park,https://s3-media3.fl.yelpcdn.com/bphoto/GvhHPX...,False,https://www.yelp.com/biz/mirai-sushi-wicker-pa...,534,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 41.90349, 'longitude': -87.6782}","[delivery, pickup]",$$$,"{'address1': '2020 W Division St', 'address2':...",+17738628500,(773) 862-8500,3952.546185
999,a-k5z5l6yx9__nI0zAbkdw,potbelly-sandwich-shop-chicago-59,Potbelly Sandwich Shop,https://s3-media1.fl.yelpcdn.com/bphoto/3tw9BQ...,False,https://www.yelp.com/biz/potbelly-sandwich-sho...,67,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",2.5,"{'latitude': 41.8965288163129, 'longitude': -8...","[delivery, pickup]",$,"{'address1': '225 E Chicago Ave', 'address2': ...",+13128374896,(312) 837-4896,3425.113476


TypeError: 'list' object is not callable